## Begin by importing the proper libraries:
    1) Requests allows us to retrieve a website.

    2) bs4 turns a website into a BeautifulSoup object.

    3) openpyxl allow us to manipulate excel files.

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bsoup
import openpyxl as opx

The web address we are interested in is the collegiate cycling directory for national mountain bikers. For the sake of example, we are interested in the *school name* and the *conference*.

In the following cells, we can see how we traverse from type to type to continually create the object we need to work with. Understanding how object types change, and what you can perform with those objects, is the essence of object oriented programming - it's very important to know what we are working with at all times. I have made that explicitly in the following cells.

In [ ]:
res = req.get('https://www.nationalmtb.org/collegiate-cycling-directory/')
text = res.content

print(f'Variable "res" is of type: \t {type(res)}')
print(f'Variable "text" is of type: \t {type(text)}')

Now, we make our BeautifulSoup object, passing it a bytes object to parse with html. BeautifulSoup recognizes the html language and allows us to select items within html.

In [ ]:
psoup = bsoup(text, 'html.parser')

print(f'Variable "psoup" is of type: \t {type(psoup)}')

This is where we select exactly what we want from the document. Searching though the source code on the website (which, in Google Chrome, is an option if you right click on the webpage and click "inspect"), I was able to identify that the "td" items of the "tbody" element are precisely what I want.

As we can see, we worked our way from a Response object to a list. Lists are fundamental objects that basic python users are very familiar with. They are often one of the basics taught in foundational classes. I worked my to the list to demonstrate how clean and easy this is to do – getting a user to something all python users are comfortable working with.

In [ ]:
webpage_content = psoup.select("tbody td")
print(f'Variable "x" is of type: \t {type(webpage_content)}')
print(f'Length of the list: {len(webpage_content)}')

Below, I create a excel workbook and select the sheet onto which I will be placing the items of interest on the website. Since we’re only working with one sheet, I just select the active sheet.

In [ ]:
wb = opx.Workbook()
ws = wb.active

print(f'Variable "wb" is of type: \t {type(wb)}')
print(f'Variable "ws" is of type: \t {type(ws)}')

The final step is placing the contents of the list into the sheet itself. This is completed by iterating through the list, selecting only the items that I want (every 4th and 6th one) and placing them into columns A and B of the excel worksheet. Once the cell has run, an excel workbook called "cycle" has been created in the same directory as this notebook.

In [ ]:
i = 4
j = 6

iter_up = 1

for item in range(len(webpage_content)):
    
    #Write the content to column A and B in excel
    #Breaks the cycle when there is no more content to write
    try:
        ws[f'A{iter_up}'] = webpage_content[j].text
        ws[f'B{iter_up}'] = webpage_content[i].text
    except:
        break
    
    i += 4
    j += 4
    
    iter_up += 1
    
wb.save('cycle.xlsx')
wb.close()
print('Done')

Just for fun, I added a quick demonstration on how easy it is to then read that excel document into a dataframe and manipulate it with pandas.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_excel('cycle.xlsx', header = None)
df.rename(columns = {0: 'Conference', 1 : 'School'}, inplace = True)
df['Conference'].value_counts().sort_values().plot(kind = 'barh');
plt.title('Amount of schools in Conference');